In [27]:
import numpy as np
import os
import pandas as pd
import locale

In [28]:
locale.setlocale(locale.LC_ALL, "cs_cs")
saveState = False
patNumbers = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
pd.set_option("display.precision", 3, "styler.format.decimal", ",")

medianErrorsStages = np.zeros(shape=(12, 5)) # shape=(patients, reg_stages)
runTime = np.zeros(shape=(12, 3))
lastIterMetrics = np.zeros(shape=(12, 3))
iterations = np.zeros(shape=(12, 3))

for idx, num in enumerate(patNumbers):
    patPath = f"input_files\\pacient_{num}\\registration\\"

    preFile = np.load(f"{patPath}gradient\\pacient{num}PreErrors.npz")
    initialFile = np.load(f"{patPath}gradient\\pacient{num}InitialErrors.npz")
    gradFile = np.load(f"{patPath}gradient\\pacient{num}RegErrors.npz")
    gradLineFile = np.load(f"{patPath}gradientline\\pacient{num}RegErrors.npz")
    gradLbfFile = np.load(f"{patPath}gradientlbf\\pacient{num}RegErrors.npz")
    
    gradRegFile = np.load(f"{patPath}gradient\\pacient{num}RegParams.npz")
    gradLineRegFile = np.load(f"{patPath}gradientline\\pacient{num}RegParams.npz")
    gradLbfRegFile = np.load(f"{patPath}gradientlbf\\pacient{num}RegParams.npz")

    # median errors per patient and registration stage
    medianErrorsStages[idx, 0] = preFile["preMedians"] # before initialization
    medianErrorsStages[idx, 1] = initialFile["initialMedians"] # after initialization
    medianErrorsStages[idx, 2] = gradFile["regMedians"][-1] # gradient 
    medianErrorsStages[idx, 3] = gradLineFile["regMedians"][-1] # gradient line 
    medianErrorsStages[idx, 4] = gradLbfFile["regMedians"][-1] # lbfgs
    
    lastIterMetrics[idx, 0] = gradRegFile["metricValues"][-1]
    lastIterMetrics[idx, 1] = gradLineRegFile["metricValues"][-1]
    lastIterMetrics[idx, 2] = gradLbfRegFile["metricValues"][-1]
    
    runTime[idx, 0] = gradRegFile["execTime"]
    runTime[idx, 1] = gradLineRegFile["execTime"]
    runTime[idx, 2] = gradLbfRegFile["execTime"]
    iterations[idx, 0] = len(gradRegFile["metricValues"]) - 1
    iterations[idx, 1] = len(gradLineRegFile["metricValues"]) - 1
    iterations[idx, 2] = len(gradLbfRegFile["metricValues"]) - 1

    del(preFile, initialFile, gradFile, gradLineFile, gradLbfFile, gradRegFile,
        gradLineRegFile, gradLbfRegFile)
    

regStages = ["Před inicializací", "Po inicializaci", "Gradientní\nsestup", 
             "Gradientní sestup\nse zl. řezem", "BFGS s\nomezenou pamětí"]
regCols = ["pre", "initial", "grad", 
             "grad_line", "grad_lbf"]
regLabels = ["Před inicializací", "Po inicializaci", "Gradientní sestup", 
             "Gradientní sestup se zl. řezem", "BFGS s omezenou pamětí"]

In [29]:
medianErrorsDf = pd.DataFrame(medianErrorsStages, columns=pd.Index(regLabels),
                                       index=pd.Index(range(1, medianErrorsStages.shape[0] + 1), name="Číslo pacienta"))

medianErrorsDfStyle = medianErrorsDf[:6].T.style.format(precision=2, decimal=",").to_latex()
print(medianErrorsDfStyle)
medianErrorsDf.T

\begin{tabular}{lrrrrrr}
Číslo pacienta & 1 & 2 & 3 & 4 & 5 & 6 \\
Před inicializací & 95,20 & 99,37 & 59,65 & 77,28 & 84,56 & 125,29 \\
Po inicializaci & 102,13 & 79,78 & 23,65 & 83,10 & 20,32 & 60,21 \\
Gradientní sestup & 12,52 & 22,75 & 6,51 & 11,69 & 10,80 & 8,98 \\
Gradientní sestup se zl. řezem & 12,82 & 22,74 & 5,53 & 11,95 & 11,11 & 9,40 \\
BFGS s omezenou pamětí & 10,61 & 22,11 & 5,31 & 12,16 & 11,17 & 7,10 \\
\end{tabular}



Číslo pacienta,1,2,3,4,5,6,7,8,9,10,11,12
Před inicializací,95.205,99.369,59.651,77.284,84.555,125.291,51.334,25.339,117.979,30.762,44.756,115.841
Po inicializaci,102.127,79.781,23.646,83.099,20.323,60.212,33.765,14.480,62.515,26.047,32.824,42.759
Gradientní sestup,12.519,22.751,6.506,11.695,10.801,8.985,6.825,4.892,4.121,15.856,18.469,4.425
Gradientní sestup se zl. řezem,12.819,22.735,5.527,11.947,11.108,9.397,6.148,6.141,3.658,14.894,19.661,4.133
BFGS s omezenou pamětí,10.614,22.112,5.308,12.163,11.173,7.102,5.969,6.032,3.597,16.291,20.742,4.346


In [30]:
method_idx = np.zeros(shape=12) 
for idx, col in enumerate(medianErrorsStages[:, 2:]):
    method_idx[idx] = col.argmin()

counts = np.unique(method_idx, return_counts=True)
counts

(array([0., 1., 2.]), array([4, 2, 6], dtype=int64))

In [31]:
metricsDf = pd.DataFrame(lastIterMetrics, columns=pd.Index(regLabels[2:]),
                                    index=pd.Index(range(1, lastIterMetrics.shape[0] + 1), name="Číslo pacienta"))

metricsDfStyle = metricsDf[6:].T.style.format(precision=3, decimal=",").to_latex()
print(metricsDfStyle)
metricsDf.T

\begin{tabular}{lrrrrrr}
Číslo pacienta & 7 & 8 & 9 & 10 & 11 & 12 \\
Gradientní sestup & -0,669 & -0,625 & -0,694 & -0,626 & -0,619 & -0,662 \\
Gradientní sestup se zl. řezem & -0,676 & -0,632 & -0,703 & -0,632 & -0,624 & -0,670 \\
BFGS s omezenou pamětí & -0,676 & -0,632 & -0,704 & -0,634 & -0,629 & -0,671 \\
\end{tabular}



Číslo pacienta,1,2,3,4,5,6,7,8,9,10,11,12
Gradientní sestup,-0.643,-0.456,-0.655,-0.545,-0.622,-0.642,-0.669,-0.625,-0.694,-0.626,-0.619,-0.662
Gradientní sestup se zl. řezem,-0.644,-0.456,-0.663,-0.555,-0.629,-0.645,-0.676,-0.632,-0.703,-0.632,-0.624,-0.670
BFGS s omezenou pamětí,-0.642,-0.457,-0.664,-0.596,-0.629,-0.639,-0.676,-0.632,-0.704,-0.634,-0.629,-0.671


In [32]:
runTimeDf = pd.DataFrame(runTime, columns=pd.Index(regLabels[2:]), 
                         index=pd.Index(range(1, runTime.shape[0] + 1), name="Číslo pacienta"))
runTimeDfStyle = runTimeDf[6:].T.style.format(precision=2, decimal=",").to_latex()
print(runTimeDfStyle)
runTimeDf.T

\begin{tabular}{lrrrrrr}
Číslo pacienta & 7 & 8 & 9 & 10 & 11 & 12 \\
Gradientní sestup & 0,89 & 0,79 & 1,12 & 1,02 & 1,46 & 1,01 \\
Gradientní sestup se zl. řezem & 1,96 & 2,31 & 1,59 & 4,32 & 7,27 & 2,06 \\
BFGS s omezenou pamětí & 6,36 & 6,39 & 4,36 & 6,44 & 5,15 & 4,78 \\
\end{tabular}



Číslo pacienta,1,2,3,4,5,6,7,8,9,10,11,12
Gradientní sestup,1.528,2.108,1.139,1.636,1.043,1.205,0.888,0.795,1.119,1.019,1.465,1.013
Gradientní sestup se zl. řezem,3.237,4.081,4.218,5.561,3.810,5.533,1.962,2.310,1.595,4.320,7.274,2.061
BFGS s omezenou pamětí,7.397,7.426,6.856,7.320,7.488,5.513,6.359,6.385,4.362,6.438,5.154,4.784


In [39]:
iterationsDf = pd.DataFrame(iterations, columns=pd.Index(regLabels[2:]),
                            index=pd.Index(range(1, iterations.shape[0] + 1), name="Číslo pacienta")).astype(int)
iterationsDfStyle = iterationsDf[6:].T.style.format(precision=2, decimal=",").to_latex()
print(iterationsDfStyle)
iterationsDf.T

\begin{tabular}{lrrrrrr}
Číslo pacienta & 7 & 8 & 9 & 10 & 11 & 12 \\
Gradientní sestup & 22 & 13 & 33 & 22 & 30 & 26 \\
Gradientní sestup se zl. řezem & 13 & 15 & 12 & 22 & 28 & 12 \\
BFGS s omezenou pamětí & 15 & 14 & 12 & 18 & 24 & 18 \\
\end{tabular}



Číslo pacienta,1,2,3,4,5,6,7,8,9,10,11,12
Gradientní sestup,78,91,26,38,20,44,22,13,33,22,30,26
Gradientní sestup se zl. řezem,21,21,21,14,19,15,13,15,12,22,28,12
BFGS s omezenou pamětí,37,27,15,28,19,19,15,14,12,18,24,18


In [34]:
with pd.ExcelWriter("graficke_vystupy\\vystupy_registrace.xlsx") as excel_writer:
    medianErrorsDf.to_excel(excel_writer, sheet_name="median_mezibodove_vzdalenosti",
                                       float_format="%.3f",
                                       columns=medianErrorsDf.columns)
    
    runTimeDf.to_excel(excel_writer, sheet_name="vypocetni_doba",
                       float_format="%.3f", 
                       columns=runTimeDf.columns)
    
    metricsDf.to_excel(excel_writer, sheet_name="vzajemna_informace",
                                  float_format="%.3f",
                                  columns=metricsDf.columns)
    iterationsDf.to_excel(excel_writer, sheet_name="pocet_iteraci",
                          columns=iterationsDf.columns)
    